# Step 2 - Database Engineering

In [1]:
#importing the dependencies
import pandas as pd

#importing Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float

In [3]:
#determine the file to import
file1 = 'Resources/hawaii_measurements_clean.csv'
file2 = 'Resources/hawaii_stations_clean.csv'

In [4]:
#reading the files with pandas
HI_measurement_df = pd.read_csv(file1)
HI_stations_df = pd.read_csv(file2)

print(HI_measurement_df.head())
print(HI_stations_df.head())

   Unnamed: 0      station        date  prcp  tobs
0           0  USC00519397  2010-01-01  0.08    65
1           1  USC00519397  2010-01-02  0.00    63
2           2  USC00519397  2010-01-03  0.00    74
3           3  USC00519397  2010-01-04  0.00    76
4           4  USC00519397  2010-01-06  0.00    73
   Unnamed: 0      station                                    name  latitude  \
0           0  USC00519397                    WAIKIKI 717.2, HI US   21.2716   
1           1  USC00513117                    KANEOHE 838.1, HI US   21.4234   
2           2  USC00514830  KUALOA RANCH HEADQUARTERS 886.9, HI US   21.5213   
3           3  USC00517948                       PEARL CITY, HI US   21.3934   
4           4  USC00518838              UPPER WAHIAWA 874.3, HI US   21.4992   

   longitude  elevation  
0  -157.8168        3.0  
1  -157.8015       14.6  
2  -157.8374        7.0  
3  -157.9751       11.9  
4  -158.0111      306.6  


In [5]:
#using the declarative base to create 2 classes, one for measurement and one for stations
Base = declarative_base()

class Measurement(Base):
    __tablename__ = 'measurement'
    
    id = Column(Integer, primary_key = True)
    station = Column(String(50))
    date = Column(String(50))
    prcp = Column(Float)
    tobs = Column(Integer)

class Station(Base):
    __tablename__ = 'station'
    
    station = Column(String, primary_key = True)
    name = Column(String(100))
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)

In [6]:
#create an engine
engine = create_engine("sqlite:///hawaii.sqlite")
engine

Engine(sqlite:///hawaii.sqlite)

In [7]:
#creating a connection to the engine called `conn`
conn = engine.connect()
conn

In [8]:
#creating the tables into the database
Base.metadata.create_all(engine)

In [9]:
engine.table_names()

['measurement', 'station']

In [10]:
#creating a session
from sqlalchemy.orm import Session
session = Session(bind=engine)

In [14]:
#adding the measurement data to the database
for i, row in HI_measurement_df.iterrows():
    data = Measurement(
                    station = row.station,
                    date = row.date,
                    prcp = row.prcp,
                    tobs = row.tobs
                    )
    session.add(data)
    session.commit()

In [16]:
#adding the station data to the database
for i, row in HI_stations_df.iterrows():
    data = Station(
                    station = row.station,
                    name = row.name,
                    latitude = row.latitude,
                    longitude = row.longitude,
                    elevation = row.elevation
                    )
    session.add(data)
    session.commit()

In [17]:
#creating a small query to check info
station_list = session.query(Station)
for station in station_list:
    print(station.__dict__)

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x00000145736136A0>, 'latitude': 21.2716, 'station': 'USC00519397', 'longitude': -157.8168, 'name': b'\x00\x00\x00\x00\x00\x00\x00\x00', 'elevation': 3.0}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x0000014573613710>, 'latitude': 21.4234, 'station': 'USC00513117', 'longitude': -157.8015, 'name': b'\x01\x00\x00\x00\x00\x00\x00\x00', 'elevation': 14.6}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x0000014573613780>, 'latitude': 21.5213, 'station': 'USC00514830', 'longitude': -157.8374, 'name': b'\x02\x00\x00\x00\x00\x00\x00\x00', 'elevation': 7.0}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x00000145736137F0>, 'latitude': 21.3934, 'station': 'USC00517948', 'longitude': -157.9751, 'name': b'\x03\x00\x00\x00\x00\x00\x00\x00', 'elevation': 11.9}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x0000014573613898>, 'latitude': 

In [18]:
measurement_list = session.query(Measurement)
for measure in measurement_list:
    print(measure.__dict__)

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x0000014573609860>, 'prcp': 0.08, 'station': 'USC00519397', 'tobs': 65, 'date': '2010-01-01', 'id': 1}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x0000014573609588>, 'prcp': 0.0, 'station': 'USC00519397', 'tobs': 63, 'date': '2010-01-02', 'id': 2}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x00000145736096D8>, 'prcp': 0.0, 'station': 'USC00519397', 'tobs': 74, 'date': '2010-01-03', 'id': 3}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x0000014573609BA8>, 'prcp': 0.0, 'station': 'USC00519397', 'tobs': 76, 'date': '2010-01-04', 'id': 4}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x0000014573609CF8>, 'prcp': 0.0, 'station': 'USC00519397', 'tobs': 73, 'date': '2010-01-06', 'id': 5}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x0000014573609C18>, 'prcp': 0.06, 'station': 'USC00519397', 'tobs': 7

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x0000014572CFE4E0>, 'prcp': 0.0, 'station': 'USC00519397', 'tobs': 79, 'date': '2014-07-10', 'id': 1592}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x0000014572CFE588>, 'prcp': 0.0, 'station': 'USC00519397', 'tobs': 80, 'date': '2014-07-11', 'id': 1593}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x0000014572CFE630>, 'prcp': 0.0, 'station': 'USC00519397', 'tobs': 81, 'date': '2014-07-12', 'id': 1594}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x0000014572CFE6D8>, 'prcp': 0.0, 'station': 'USC00519397', 'tobs': 80, 'date': '2014-07-13', 'id': 1595}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x0000014572CFE780>, 'prcp': 0.0, 'station': 'USC00519397', 'tobs': 81, 'date': '2014-07-14', 'id': 1596}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x0000014572CFE828>, 'prcp': 0.0, 'station': 'USC0051939

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x0000014573000978>, 'prcp': 0.04, 'station': 'USC00513117', 'tobs': 72, 'date': '2012-12-16', 'id': 3783}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x0000014573000A20>, 'prcp': 0.02, 'station': 'USC00513117', 'tobs': 72, 'date': '2012-12-17', 'id': 3784}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x0000014573000AC8>, 'prcp': 0.2, 'station': 'USC00513117', 'tobs': 71, 'date': '2012-12-18', 'id': 3785}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x0000014573000B70>, 'prcp': 0.24, 'station': 'USC00513117', 'tobs': 70, 'date': '2012-12-19', 'id': 3786}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x0000014573000C18>, 'prcp': 0.08, 'station': 'USC00513117', 'tobs': 71, 'date': '2012-12-20', 'id': 3787}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x0000014573000CC0>, 'prcp': 0.06, 'station': 'USC00

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x0000014573315C18>, 'prcp': 0.0, 'station': 'USC00514830', 'tobs': 70, 'date': '2011-01-10', 'id': 5731}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x0000014573315CC0>, 'prcp': 0.61, 'station': 'USC00514830', 'tobs': 58, 'date': '2011-01-11', 'id': 5732}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x0000014573315D68>, 'prcp': 0.54, 'station': 'USC00514830', 'tobs': 66, 'date': '2011-01-12', 'id': 5733}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x0000014573315E10>, 'prcp': 3.91, 'station': 'USC00514830', 'tobs': 68, 'date': '2011-01-13', 'id': 5734}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x0000014573315EB8>, 'prcp': 0.17, 'station': 'USC00514830', 'tobs': 74, 'date': '2011-01-14', 'id': 5735}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x0000014573315F60>, 'prcp': 0.0, 'station': 'USC005

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x0000014574A40C18>, 'prcp': 0.0, 'station': 'USC00517948', 'tobs': 65, 'date': '2011-02-02', 'id': 7795}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x0000014574A40CC0>, 'prcp': 0.0, 'station': 'USC00517948', 'tobs': 68, 'date': '2011-02-03', 'id': 7796}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x0000014574A40D68>, 'prcp': 0.0, 'station': 'USC00517948', 'tobs': 64, 'date': '2011-02-04', 'id': 7797}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x0000014574A40E10>, 'prcp': 0.0, 'station': 'USC00517948', 'tobs': 69, 'date': '2011-02-07', 'id': 7798}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x0000014574A40EB8>, 'prcp': 1.6, 'station': 'USC00517948', 'tobs': 69, 'date': '2011-02-08', 'id': 7799}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x0000014574A40F60>, 'prcp': 0.0, 'station': 'USC0051794

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x0000014574CCB978>, 'prcp': 0.0, 'station': 'USC00519523', 'tobs': 73, 'date': '2010-10-19', 'id': 9807}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x0000014574CCBA20>, 'prcp': 0.0, 'station': 'USC00519523', 'tobs': 74, 'date': '2010-10-20', 'id': 9808}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x0000014574CCBAC8>, 'prcp': 0.0, 'station': 'USC00519523', 'tobs': 77, 'date': '2010-10-21', 'id': 9809}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x0000014574CCBB70>, 'prcp': 0.0, 'station': 'USC00519523', 'tobs': 76, 'date': '2010-10-22', 'id': 9810}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x0000014574CCBC18>, 'prcp': 0.0, 'station': 'USC00519523', 'tobs': 77, 'date': '2010-10-23', 'id': 9811}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x0000014574CCBCC0>, 'prcp': 0.0, 'station': 'USC0051952

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x0000014574F28D68>, 'prcp': 0.0, 'station': 'USC00519523', 'tobs': 67, 'date': '2016-02-29', 'id': 11685}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x0000014574F28E10>, 'prcp': 0.33, 'station': 'USC00519523', 'tobs': 73, 'date': '2016-03-01', 'id': 11686}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x0000014574F28EB8>, 'prcp': 0.0, 'station': 'USC00519523', 'tobs': 70, 'date': '2016-03-02', 'id': 11687}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x0000014574F28F60>, 'prcp': 0.0, 'station': 'USC00519523', 'tobs': 68, 'date': '2016-03-03', 'id': 11688}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x0000014574F30048>, 'prcp': 0.0, 'station': 'USC00519523', 'tobs': 75, 'date': '2016-03-04', 'id': 11689}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x0000014574F300F0>, 'prcp': 0.0, 'station': 'USC0

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x0000014575146E10>, 'prcp': 0.28, 'station': 'USC00519281', 'tobs': 70, 'date': '2013-03-29', 'id': 13366}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x0000014575146EB8>, 'prcp': 0.18, 'station': 'USC00519281', 'tobs': 72, 'date': '2013-03-30', 'id': 13367}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x0000014575146F60>, 'prcp': 0.04, 'station': 'USC00519281', 'tobs': 62, 'date': '2013-03-31', 'id': 13368}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x000001457514F048>, 'prcp': 0.0, 'station': 'USC00519281', 'tobs': 68, 'date': '2013-04-01', 'id': 13369}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x000001457514F0F0>, 'prcp': 0.02, 'station': 'USC00519281', 'tobs': 66, 'date': '2013-04-02', 'id': 13370}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x000001457514F198>, 'prcp': 0.0, 'station': 'U

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.
